![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

In [1]:
import sys
print(sys.executable)

C:\Users\dhoyoso\.conda\envs\gpu_tenv\python.exe


In [2]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

C:\Users\dhoyoso\.conda\envs\gpu_tenv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\dhoyoso\.conda\envs\gpu_tenv\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\dhoyoso\.conda\envs\gpu_tenv\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
import numpy, pandas
numpy.__version__
pandas.__version__

'1.3.5'

# EXPERIMENTO CON GLOVE EMBEDDINGS

In [4]:
import re
import nltk
import pandas as pd
import os
from keras import backend as K
import numpy as np
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import r2_score, roc_auc_score
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MultiLabelBinarizer

# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dhoyoso\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dhoyoso\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dhoyoso\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dhoyoso\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
# Definición de variable de interés (y)
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])
y_genres

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0]])

In [6]:
# Descargar recursos de NLTK
nltk.download('stopwords')
nltk.download('wordnet')

# Crear objeto para lematización
lemmatizer = WordNetLemmatizer()

# Obtener stopwords en inglés
stopwords = set(stopwords.words('english'))

# Preprocesar los textos
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    
    # Eliminar stopwords
    text = ' '.join(word for word in text.split() if word not in stopwords)
    
    # Lematización
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())
    
    # Eliminar caracteres especiales
    text = re.sub(r'[^a-zA-Z ]+', '', text)
    
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dhoyoso\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dhoyoso\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
# Aplicar preprocesamiento a la matriz de sinopsisarra
dataTraining['sinopsis'] = dataTraining.apply(lambda row: row['title'] + '; ' + row['plot'], axis=1)
dataTraining['sinopsis_processed'] = dataTraining['sinopsis'].apply(preprocess_text)

dataTraining['sinopsis_processed']

3107    most story single father take eight year  old ...
900     serial killer serial killer decides teach secr...
6724    womans face sweden  female blackmailer disfigu...
4704    executive suite friday afternoon new york  pre...
2582    narrow margin los angeles  editor publishing h...
                              ...                        
8417    family wedding  marriage  wedding    lesson nu...
1592    conan destroyer wandering barbarian  conan  al...
1723    kismet like tale spun scheherazade  kismet fol...
7605    secret nimh mr  brisby  widowed mouse  life ci...
215     tinker bell lost treasure tinker bell journey ...
Name: sinopsis_processed, Length: 7895, dtype: object

In [8]:
from keras.preprocessing.text import Tokenizer
vocab_size = 400000
oov_token = "<OOV>"
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)

# Realizar la partición de los datos
X_train, X_test, y_train_genres, y_test_genres = train_test_split(dataTraining['sinopsis_processed'], y_genres, test_size=0.33, random_state=42)

# Tokenizar los datos
tokenizer.fit_on_texts(X_train)

In [9]:
word_index = tokenizer.word_index

In [10]:
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [11]:
X_test_sequences[0:10]

[[2721,
  5614,
  3794,
  795,
  761,
  12364,
  925,
  6367,
  3180,
  1168,
  316,
  303,
  14,
  9999,
  3551,
  2889,
  1762,
  925,
  62,
  1,
  324,
  13,
  21,
  1153,
  272,
  1,
  4285,
  737,
  3619,
  536,
  1257],
 [19,
  7862,
  24,
  69,
  689,
  463,
  814,
  1,
  764,
  1,
  26577,
  837,
  639,
  2673,
  206,
  6,
  6244,
  4141,
  1889,
  1,
  344,
  101,
  851,
  5968,
  4552],
 [859,
  398,
  13931,
  384,
  982,
  118,
  2079,
  1846,
  1317,
  14817,
  10,
  221,
  9066,
  13931,
  150,
  158,
  720,
  203,
  7,
  67,
  51,
  1301,
  24,
  55,
  9,
  13931,
  14923,
  274,
  2158,
  1196,
  1297,
  729,
  5,
  720,
  482,
  375,
  2268,
  388,
  3709,
  3321,
  1,
  1645,
  1962,
  13931,
  248,
  140,
  947,
  17,
  2260,
  1768,
  1301,
  2110,
  150,
  79,
  277,
  13931,
  1597,
  4,
  2500,
  247,
  5,
  27,
  6513,
  887,
  1317,
  301,
  9066,
  480,
  52,
  28514,
  304,
  743,
  15],
 [101,
  6642,
  1149,
  3622,
  1,
  5883,
  7176,
  631,
  1,
  1,
  1

In [12]:
max_length = 300
padding_type='post'
truncation_type='post'

In [13]:
from keras.preprocessing.sequence import pad_sequences

X_test_padded = pad_sequences(X_test_sequences,maxlen=max_length, 
                               padding=padding_type, truncating=truncation_type)
X_train_padded = pad_sequences(X_train_sequences,maxlen=max_length, padding=padding_type, 
                       truncating=truncation_type)

In [14]:
X_train_padded

array([[ 1645, 17990,    22, ...,     0,     0,     0],
       [   37,  4900,   692, ...,     0,     0,     0],
       [ 6690, 13583,     2, ...,     0,     0,     0],
       ...,
       [ 3413, 13362,  2846, ...,     0,     0,     0],
       [ 1473,  2750,  7230, ...,     0,     0,     0],
       [  271, 10508,    34, ...,     0,     0,     0]])

In [15]:
import numpy as np
embeddings_index = {}
f = open("glove.6B.300d.txt", encoding="UTF-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [16]:
embedding_matrix = np.zeros((len(word_index) + 1, max_length))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten, Input, Conv1D, MaxPooling1D, Concatenate
from keras.models import Model

embedding_layer = Embedding(input_dim=len(word_index) + 1,
                            output_dim=max_length,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)

# Obtener el número de géneros
num_genres = y_train_genres.shape[1]

model = Sequential([
    embedding_layer,
    Bidirectional(LSTM(150, return_sequences=True)), 
    Bidirectional(LSTM(150)),
    Dense(128, activation='relu'),
    Dense(num_genres, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 300)          8935200   
_________________________________________________________________
bidirectional (Bidirectional (None, 300, 300)          541200    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300)               541200    
_________________________________________________________________
dense (Dense)                (None, 128)               38528     
_________________________________________________________________
dense_1 (Dense)              (None, 24)                3096      
Total params: 10,059,224
Trainable params: 1,124,024
Non-trainable params: 8,935,200
_________________________________________________________________


In [18]:
model.compile(loss='binary_crossentropy',optimizer='adamax', metrics=['accuracy'])

In [30]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
%load_ext tensorboard

early_stopping = EarlyStopping(monitor="val_accuracy", patience = 5)

log_folder = 'logs'
callbacks = [
            early_stopping,
            TensorBoard(log_dir=log_folder)
            ]
num_epochs = 600
history = model.fit(X_train_padded, y_train_genres, batch_size = 256, epochs=num_epochs, validation_data=(X_test_padded, y_test_genres),callbacks=callbacks)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Epoch 1/600
21/21 [==============================] - 7s 312ms/step - loss: 0.1490 - accuracy: 0.4944 - val_loss: 0.2238 - val_accuracy: 0.4133
Epoch 2/600
21/21 [==============================] - 6s 308ms/step - loss: 0.1477 - accuracy: 0.4876 - val_loss: 0.2254 - val_accuracy: 0.4098
Epoch 3/600
21/21 [==============================] - 7s 335ms/step - loss: 0.1437 - accuracy: 0.4954 - val_loss: 0.2254 - val_accuracy: 0.3960
Epoch 4/600
21/21 [==============================] - 7s 338ms/step - loss: 0.1407 - accuracy: 0.4903 - val_loss: 0.2292 - val_accuracy: 0.4217
Epoch 5/600
21/21 [==============================] - 7s 335ms/step - loss: 0.1393 - accuracy: 0.5026 - val_loss: 0.2294 - val_accuracy: 0.3937
Epoch 6/600
21/21 [==============================] - 7s 342ms/step - loss: 0.1366 - accuracy: 0.4959 - val_loss: 0.2305 - val_accuracy: 0.4048
Epoch 7/600
21/21 [==============================] -

In [31]:
loss, accuracy = model.evaluate(X_test_padded,y_test_genres)
print('Test accuracy :', accuracy)

82/82 [==============================] - 3s 37ms/step - loss: 0.2379 - accuracy: 0.3588
Test accuracy : 0.3587874174118042


In [21]:
dataTesting['sinopsis'] = dataTesting.apply(lambda row: row['title'] + '; ' + row['plot'], axis=1)
dataTesting['sinopsis_processed'] = dataTesting['sinopsis'].apply(preprocess_text)

X_test_submission_sequences = tokenizer.texts_to_sequences(dataTesting['sinopsis_processed'])

X_test_submission_padded = pad_sequences(X_test_submission_sequences,maxlen=max_length, 
                               padding=padding_type, truncating=truncation_type)

In [22]:
# X_test_dtm = X_test_dtm.toarray()
# Predecir las etiquetas de género para el conjunto de prueba
y_pred_test_genres = model.predict(X_test_submission_padded)

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)
res.to_csv('pred_genres_text_RF.csv', index_label='ID')
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.134164,0.087074,0.024878,0.047566,0.388742,0.257270,0.042809,0.568447,0.063573,0.087191,...,0.036734,0.106583,0.001113,0.297827,0.056587,0.010596,0.025261,0.204091,0.038663,0.025945
4,0.260267,0.150722,0.026526,0.051781,0.286744,0.230790,0.063613,0.533125,0.071919,0.067926,...,0.037149,0.114418,0.002079,0.190355,0.101284,0.014428,0.034403,0.375410,0.063920,0.039844
5,0.287102,0.155477,0.029589,0.053116,0.287544,0.239995,0.067428,0.524082,0.073107,0.069455,...,0.040298,0.128406,0.002459,0.188330,0.109945,0.016211,0.038031,0.409006,0.070264,0.045114
6,0.251559,0.142144,0.027257,0.050830,0.286104,0.246902,0.063232,0.555720,0.067726,0.068071,...,0.037316,0.117346,0.001891,0.195095,0.096194,0.014373,0.033095,0.381122,0.065452,0.038568
7,0.204960,0.123655,0.027824,0.053675,0.321454,0.231040,0.052801,0.545847,0.070962,0.079493,...,0.041755,0.129533,0.001747,0.232695,0.080112,0.013091,0.030983,0.308395,0.053565,0.036251


# Calibracion

In [23]:
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras import metrics
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.callbacks import EarlyStopping
from keras import backend as K
#from livelossplot import PlotLossesKeras

In [24]:
# Definición de parámetros y sus valores sobre los que se va a calibrar
nn_params = {
    'output_activation': ['sigmoid', 'softmax'],
    #'neurons_lstm':[64, 128, 256, 512],
    #'neurons_dense':[64, 128, 256, 512],
    'patience':[2, 5],
    'loss':['binary_crossentropy', 'categorical_crossentropy']
}

# Definición de función que crea una red neuronal a partir de diferentes parámetros (nn_model_params)
# En esta función se consideran 8 parámetos a calibrar, sin embargo se pueden agregar o quitar tantos como lo consideren pertinente
def nn_model_params(output_activation,
                    patience,
                    loss):
    
    K.clear_session()
    

    # Definición red neuronal con la función Sequential()
    # Definición de las capas de la red con el número de neuronas y la función de activación definidos en la función nn_model_params
    model = Sequential([
        embedding_layer,
        Bidirectional(LSTM(150, return_sequences=True)), 
        Bidirectional(LSTM(150)),
        Dense(128, activation='relu'),
        Dense(24, activation=output_activation)
    ])

    # Definición de función de perdida con parámetros definidos en la función nn_model_params
    model.compile(optimizer = 'adamax', loss=loss, metrics=['accuracy'])
    
    # Definición de la función EarlyStopping con parámetro definido en la función nn_model_params
    early_stopping = EarlyStopping(monitor="val_accuracy", patience = patience)

    # Entrenamiento de la red neuronal con parámetros definidos en la función nn_model_params
    model.fit(X_train_padded, y_train_genres,
              validation_data = (X_test_padded, y_test_genres),
              epochs = 600,
              batch_size=256,
              callbacks=[early_stopping],
              verbose=True
             )

     
    return model

In [32]:
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# Definición de red neuronal usando el wrapper KerasRegressor y usando como argumento build_fn en la función nn_model_params
nn_model = KerasClassifier(build_fn=nn_model_params, verbose=0)

# Definición método GridSearch para la calibración de parámetros definidos en nn_params
rs = RandomizedSearchCV(nn_model, param_distributions=nn_params, n_iter=10, cv = 2)
rs.fit(X_train_padded, y_train_genres)

print('Los mejores parametros segun Randomnized Search:', rs.best_params_)

C:\Users\dhoyoso\.conda\envs\gpu_tenv\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Epoch 1/600
21/21 [==============================] - 11s 342ms/step - loss: 0.4162 - accuracy: 0.1267 - val_loss: 0.3016 - val_accuracy: 0.3120
Epoch 2/600
21/21 [==============================] - 6s 310ms/step - loss: 0.2939 - accuracy: 0.2276 - val_loss: 0.2958 - val_accuracy: 0.1926
Epoch 3/600
21/21 [==============================] - 6s 295ms/step - loss: 0.2911 - accuracy: 0.2002 - val_loss: 0.2931 - val_accuracy: 0.1926
Epoch 1/600
21/21 [==============================] - 12s 349ms/step - loss: 0.4475 - accuracy: 0.1558 - val_loss: 0.3029 - val_accuracy: 0.2214
Epoch 2/600
21/21 [==============================] - 7s 323ms/step - loss: 0.2944 - accuracy: 0.2040 - val_loss: 0.2949 - val_accuracy: 0.1926
Epoch 3/600
21/21 [==============================] - 7s 343ms/step - loss: 0.2904 - accuracy: 0.2002 - val_loss: 0.2911 - val_accuracy: 0.1926
Epoch 1/600
21/21 [==============================] - 12s 357ms/step - loss: 0.4511 - accuracy: 0.1588 - val_loss: 0.3065 - val_accuracy: 0.1

21/21 [==============================] - 8s 365ms/step - loss: 0.2132 - accuracy: 0.3798 - val_loss: 0.2302 - val_accuracy: 0.3561
Epoch 20/600
21/21 [==============================] - 8s 364ms/step - loss: 0.2085 - accuracy: 0.3961 - val_loss: 0.2284 - val_accuracy: 0.3319
Epoch 21/600
21/21 [==============================] - 8s 370ms/step - loss: 0.2053 - accuracy: 0.3925 - val_loss: 0.2278 - val_accuracy: 0.3580
Epoch 1/600
21/21 [==============================] - 12s 367ms/step - loss: 0.4476 - accuracy: 0.0297 - val_loss: 0.3091 - val_accuracy: 0.1926
Epoch 2/600
21/21 [==============================] - 7s 346ms/step - loss: 0.2970 - accuracy: 0.2002 - val_loss: 0.2963 - val_accuracy: 0.1926
Epoch 3/600
21/21 [==============================] - 7s 357ms/step - loss: 0.2922 - accuracy: 0.2002 - val_loss: 0.2951 - val_accuracy: 0.1926
Epoch 1/600
21/21 [==============================] - 12s 360ms/step - loss: 0.3931 - accuracy: 0.2053 - val_loss: 0.2993 - val_accuracy: 0.1926
Epoch 2

Epoch 1/600
21/21 [==============================] - 12s 365ms/step - loss: 9.7305 - accuracy: 0.2118 - val_loss: 14.6878 - val_accuracy: 0.3120
Epoch 2/600
21/21 [==============================] - 7s 337ms/step - loss: 15.9351 - accuracy: 0.2276 - val_loss: 19.5675 - val_accuracy: 0.1926
Epoch 3/600
21/21 [==============================] - 8s 365ms/step - loss: 21.5420 - accuracy: 0.2002 - val_loss: 26.6291 - val_accuracy: 0.1926
Epoch 1/600
21/21 [==============================] - 12s 358ms/step - loss: 9.7332 - accuracy: 0.1938 - val_loss: 14.0449 - val_accuracy: 0.3120
Epoch 2/600
21/21 [==============================] - 7s 331ms/step - loss: 16.0279 - accuracy: 0.2169 - val_loss: 18.6681 - val_accuracy: 0.1926
Epoch 3/600
21/21 [==============================] - 7s 350ms/step - loss: 21.0935 - accuracy: 0.2002 - val_loss: 24.5766 - val_accuracy: 0.1926
Epoch 1/600
21/21 [==============================] - 12s 362ms/step - loss: 9.7778 - accuracy: 0.1997 - val_loss: 13.6433 - val_ac

In [27]:
# Desempeño de modelo con los parametros de Randomized Search
# {'patience': 5, 'output_activation': 'sigmoid', 'loss': 'binary_crossentropy'}
model = nn_model_params(
                        patience=5,
                        output_activation='sigmoid',
                        loss='binary_crossentropy'
                       )

Epoch 1/600
21/21 [==============================] - 12s 340ms/step - loss: 0.4257 - accuracy: 0.1089 - val_loss: 0.3026 - val_accuracy: 0.1926
Epoch 2/600
21/21 [==============================] - 6s 295ms/step - loss: 0.2941 - accuracy: 0.2002 - val_loss: 0.2951 - val_accuracy: 0.1926
Epoch 3/600
21/21 [==============================] - 6s 298ms/step - loss: 0.2906 - accuracy: 0.2108 - val_loss: 0.2921 - val_accuracy: 0.1930
Epoch 4/600
21/21 [==============================] - 6s 305ms/step - loss: 0.2858 - accuracy: 0.2486 - val_loss: 0.2849 - val_accuracy: 0.3135
Epoch 5/600
21/21 [==============================] - 7s 320ms/step - loss: 0.2761 - accuracy: 0.3038 - val_loss: 0.2743 - val_accuracy: 0.3365
Epoch 6/600
21/21 [==============================] - 7s 320ms/step - loss: 0.2654 - accuracy: 0.3466 - val_loss: 0.2704 - val_accuracy: 0.2556
Epoch 7/600
21/21 [==============================] - 7s 327ms/step - loss: 0.2579 - accuracy: 0.3284 - val_loss: 0.2581 - val_accuracy: 0.328

In [28]:
loss, accuracy = model.evaluate(X_test_padded,y_test_genres)
print('Test accuracy :', accuracy)

82/82 [==============================] - 3s 36ms/step - loss: 0.2188 - accuracy: 0.3918
Test accuracy : 0.39178818464279175


In [32]:
# Guardar el Modelo
model.save('final_model.h5')

In [37]:
import pickle

# Guardar el tokenizador
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [36]:
import keras

# Recrea exactamente el mismo modelo solo desde el archivo
new_model = keras.models.load_model('final_model.h5')

In [38]:
with open('tokenizer.pickle', 'rb') as handle:
    new_tokenizer = pickle.load(handle)

In [72]:
test = ['horror movie']

X_test_seq = new_tokenizer.texts_to_sequences(test)

X_test_pad = pad_sequences(X_test_seq,maxlen=max_length, 
                               padding=padding_type, truncating=truncation_type)

preds = new_model.predict(X_test_pad)

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']


prediccion = pd.DataFrame(preds, columns=cols)
prediccion.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
0,0.109952,0.024771,0.036794,0.004108,0.290189,0.042824,0.203385,0.027371,0.01051,0.072733,...,0.006351,0.021919,0.000673,0.013873,0.085907,0.031826,0.001152,0.126808,0.002138,0.000535


In [73]:
cols[np.argmax(preds[0])]

'p_Horror'